In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import pickle
from scipy.stats import skew, kurtosis, pearsonr
from scipy.signal import butter, welch, filtfilt, resample
import copy
import time
import datetime
from multiprocessing.dummy import Pool as ThreadPool
from PreprocessFcns import *

%matplotlib inline

In [ ]:
# Set path to folder containing Subject Records
path = r'//FS2.smpp.local\\RTO\\CIS-PD Study\MJFF Curation\Finalized Dataset'
# Set path to Destination Folder
dest = r'//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Record Correlation'
#---------------------------------------------------------------------------------------------------------

In [ ]:
# Medication Reports
table_med = 'Table10.csv'
dest_ext_med = 'Medication Reports'
file_name_med = 'med_timepoints.csv'

# Symptom Reports
table_symt = 'Table11.csv'
dest_ext_symt = 'Symptom Reports'
file_name_symt = 'symt_timepoints.csv'

# Diaries
table_diar = 'Table12.csv'
dest_ext_diar = 'Diaries'
file_name_diar = 'diar_timepoints.csv'

In [ ]:
### PROCESSER TO ITERATE THROUGH EACH WATCH DATA FILE AND BIN DATA POINT TO RELEVANT RECORDS

# Load Consolidated DataFrames
timepoints_med = pd.read_csv(os.path.join(dest, dest_ext_med, file_name_med))
timepoints_symt = pd.read_csv(os.path.join(dest, dest_ext_symt, file_name_symt))
timepoints_diar = pd.read_csv(os.path.join(dest, dest_ext_diar, file_name_diar))

# Group Record Types to Go Over All at Once
records = [timepoints_med, timepoints_symt, timepoints_diar]

# Create List of the Watch Data Files in the Table8 Directory (not including the subdirectory)
watch_dir = [f for f in os.listdir(os.path.join(path, 'Table8')) if os.path.isfile(os.path.join(path, 'Table8', f))]

#Create a DataFrame for Each Record TimeStamp? - So Each Watch Data Point can be Binned while Iterating if Necessary?
for record in records:
    for entry in range(len(record)):
        entry_watch = pd.DataFrame(columns = ['SubjID', 'Timestamp', 'X', 'Y', 'Z'])
        entry_watch.to_csv(os.path.join(dest, dest_ext, str(record.loc[entry, 'SubjID']) + ' ' + 
                                        str(record.loc[entry, 'Timestamp'])[:13] + str(record.loc[entry, 'Timestamp'])[14:16] +
                                        str(record.loc[entry, 'Timestamp'])[17:] + '.csv'), index = False)

for file in watch_dir:
    
    # Read Watch Acc File in Chunks
    watch_file_chunk = pd.read_csv(os.path.join(path, 'Table8', file), chunksize = 100000)
    
    # Sequential Indices
    i = 0
    # Changing Index of Chunk
    c = 0
    
    # Look at One Chunk at a Time
    for chunk in watch_file_chunk:
        
        for acc in range(len(chunk)):
            
            for record in records:
                
                if record == timepoints_med:
                    
                    # Iterate Through Medication Records
                    for entry in range(len(record)):
                    
                        if (pd.Timestamp(chunk.loc[c+acc, 'Timestamp']) >= pd.Timestamp(record.loc[entry, 'Timestamp']) + 
                            pd.Timedelta('-30 min') and pd.Timestamp(chunk.loc[c+acc, 'Timestamp']) <= 
                            pd.Timestamp(record.loc[entry, 'Timestamp']) + pd.Timedelta('30 min')):
            
                else:
                    
                    # Iterate Through Medication Records
                    for entry in range(len(record)):
                
                        if (pd.Timestamp(chunk.loc[c+acc, 'Timestamp']) >= pd.Timestamp(record.loc[entry, 'Timestamp']) + 
                            pd.Timedelta('-30 min') and pd.Timestamp(chunk.loc[c+acc, 'Timestamp']) <= 
                            pd.Timestamp(record.loc[entry, 'Timestamp'])):